In [15]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import requests
import json
import pprint

# creating full urls from base and tails that do not appear in html of main page (allows us to avoid clicking through to sub-urls)
hunt_code = []
short_url = 'https://dwrapps.utah.gov/huntboundary/HaNumber'
hunt_nums = list(range(3000,3129))  #hard code list of limited entry hunt numbers
for each in hunt_nums:
    string = str(each)   # change each hunt number int to str for concatenation purposes
    letters = '?hn=EB' + string   # concatenate hunt letter abbreviations with hunt numbers to create full hunt code
    hunt_code.append(short_url + letters)

# creating empty lists of parameters of interests for later population
hunt_number = []
hunt_type = []
species = []
unit_name = []
weapon_type = []
season_dates = []
percent_harvest = []
pop_obj = []
current_pop = []
age_obj = []
current_age = []

# extracting data from each hunt url
for i in hunt_code:
    res = requests.get(i)
    hunt = json.loads(res.text) 

    ### huntBios key
    res.headers['content-type']
    pagedata = res.content.decode("utf-8")
    hunt_num_json = json.loads(pagedata)
    huntBios = hunt_num_json['huntBios'][0]
    man_obj = huntBios['MANAGEMENT_OBJ']
    split = man_obj.split('~')
    percent_harvest.append(split[0])
    pop_obj.append(split[1])
    current_pop.append(split[2])
    age_obj.append(split[5])
    current_age.append(split[6])
    

    ### huntMaster key
    huntMaster = hunt_num_json['huntMaster']
    # pprint.pprint(huntMaster)
    unit_name.append(huntMaster['HUNT_NAME'])
    hunt_number.append(huntMaster['HUNT_NBR'])
    hunt_type.append(huntMaster['HUNT_TYPE'])
    species.append(huntMaster['SPECIES'])
    weapon_type.append(huntMaster['WEAPON'])


    ### huntYears key
    huntYears = hunt_num_json['huntYears']
    huntYears0 = (huntYears[0])
    szn = huntYears0['SEASON_DATE_TEXT']
    season_dates.append(szn)
    
df=pd.DataFrame()
df['Unit Name']=unit_name
df['Hunt Number']=hunt_number
df['Type']=hunt_type
df['Species']=species
df['Season Dates']=season_dates
df['Weapon']=weapon_type
df['% Harvest Success']=percent_harvest
df['Pop Objective']=pop_obj
df['Current Pop']=current_pop
df['Age Objective']=age_obj
df['Current Age']=current_age


df['% Harvest Success'].astype(str).astype(float)
# df['Pop Objective'].astype(str).astype(int)
# df['Current Pop'].astype(str).astype(int)
# df['Age Objective'].astype(str).astype(float)
# df['Current Age'].astype(str).astype(float)
# df.describe()
df.dtypes


# print(unit_name)
# print(hunt_number)
# print(hunt_type)
# print(species)
# print(weapon_type)
# print(percent_harvest)
# print(pop_obj)
# print(current_pop)
# print(age_obj)
# print(current_age)
# print(season_dates)

Unit Name            object
Hunt Number          object
Type                 object
Species              object
Season Dates         object
Weapon               object
% Harvest Success    object
Pop Objective        object
Current Pop          object
Age Objective        object
Current Age          object
dtype: object